# Alguns arquivos de salinidade em situ vieram conrompidos, para não perder dados, aqui está o processo que estou fazendo para ajeitar o aquivo

##1. Importando bibliotecas necessárias

In [2]:
import subprocess
import netCDF4
import pandas as pd

###1.1 Crie as pastas de apoio

In [3]:
subprocess.run(['mkdir','/content/Passo1'])
subprocess.run(['mkdir','/content/Passo2'])
subprocess.run(['mkdir','/content/Passo3'])

CompletedProcess(args=['mkdir', '/content/Passo3'], returncode=0)

###1.2 Extrair o arquivo comprimido do drive para o colab


In [16]:
subprocess.run(['unrar','x','/content/drive/MyDrive/tsg_2003-2010.rar'])

CompletedProcess(args=['unrar', 'x', '/content/drive/MyDrive/tsg_2003-2010.rar'], returncode=0)

##2. Primeiro vou converter os csv par arquivos que podem ser lidos no excel, trocando a (,) por (;) e o (.) por (,)

###2.1 Resgantando todos os arquivos e os tranformando em um array de nome de arquivos

In [22]:
files = subprocess.getoutput(['ls','/content/'])
files = files.split()
files.remove('drive')
files.remove('Passo1')
files.remove('Passo2')
files.remove('Passo3')
files.remove('sample_data')

###2.2 Fazendo a mudança de (,) para (;)

In [23]:
for fi in files:
  fin = open(fi, "rt")
  fout = open('/content/Passo1/'+fi, "wt")
  for line in fin:
	    fout.write(line.replace(',', ';'))
fin.close()
fout.close()

###2.2 Fazendo a mudança de (.) para (,)

In [24]:
for fi in files:
  fin = open('/content/Passo1/'+fi, "rt")
  fout = open('/content/Passo2/'+fi, "wt")
  for line in fin:
	    fout.write(line.replace('.', ','))
fin.close()
fout.close()

###2.3. O grande problema dos arquivos e que a maioria explode o numero de variaveis, para isso, vou acrescentar 7 (;) no final do arquivo para que ele junte de forma correta todas as variaveis

In [25]:
primeiraLinha = 0
for fi in files:
  primeiraLinha = 0
  fin = open('/content/Passo2/'+fi, "rt")
  fout = open('/content/Passo3/'+fi, "wt")
  for line in fin:
    if primeiraLinha == 0:
	    fout.write(line.replace('\n', ';;;;;;;\n'))
    else:
      fout.write(line)
    primeiraLinha = primeiraLinha+1
fin.close()
fout.close()

##3. Feito isso, carregue todos os documentos para podermos criar nosso df pandas unico do ano


> OBS: Não se preocupe com arquivos em branco ou repetidos, com um dataFrame pandas a gnt consegue lidar com isso na parte de pré processamento

> Parte 1: Agora crie um unico arquivo para todos os anos apenas com as colunas necessárias, caso não tenha a coluna, ele vai pular o df





In [ ]:
dx = pd.DataFrame(data=None)
for fi in files:
  try:
    df = pd.read_csv('/content/Passo3/'+fi,
                   sep=';',
                   engine='python',
                   usecols=[
                            'DATE (yyyy-mm-ddThh:mi:ssZ)','LATITUDE (degree_north)',
                            'LONGITUDE (degree_east)','PSAL LEVEL1 (psu)'])
    dx = pd.concat([dx,df])
  except: 
    pass

> Parte 2: Veja se ta tudo ok

In [13]:
dx

,DATE (yyyy-mm-ddThh:mi:ssZ),LATITUDE (degree_north),LONGITUDE (degree_east),PSAL LEVEL1 (psu)
0,2008-08-19T00:00:00Z,"10,2482","-79,4385",NaN
1,2008-08-19T00:05:00Z,"10,2752","-79,4232",NaN
2,2008-08-19T00:10:00Z,"10,3022","-79,4083",NaN
3,2008-08-19T00:15:00Z,"10,3295","-79,3938",NaN
4,2008-08-19T00:20:00Z,"10,3567","-79,3795",NaN
...,...,...,...,...
5379,2004-08-06T13:00:00Z,"39,8800","-11,1900","36,039"
5380,2004-08-06T13:05:00Z,"39,9000","-11,1800","36,014"
5381,2004-08-06T13:10:00Z,"39,9300","-11,1700","36,006"
5382,2004-08-06T13:15:00Z,"39,9600","-11,1600","35,987"


> Parte 3: Agora salve o dataFrame

>> Para bottles

In [14]:
dx.to_csv('/content/drive/MyDrive/bottles.csv',sep=';')

>> Para tsg

In [ ]:
dx.to_csv('/content/drive/MyDrive/Tsg_xxx.csv',sep=';')

##4. Por ultimo, limpe as pastas

In [15]:
for fi in files:
   subprocess.run(['rm','/content/'+fi])
   subprocess.run(['rm','/content/Passo1/'+fi])
   subprocess.run(['rm','/content/Passo2/'+fi])
   subprocess.run(['rm','/content/Passo3/'+fi])